In [ ]:
import pandas as pd
import ast
from ast import literal_eval
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from matplotlib import rcParams
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm
from collections import Counter
import os.path
from os import path

import sys
import networkx as nx
import urllib
import numpy as nр
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
from community import community_louvain
import pandas as pd
from IPython.display import display, HTML, display_pretty
import os
%matplotlib inline

In [ ]:
def read_data(path, sep):
    df = pd.read_csv(path, sep=sep)
    df.loc[:, 'AA'] = df['AA'].apply(lambda x: ast.literal_eval(x))
    df.loc[:, 'W'] = df['W'].apply(lambda x: ast.literal_eval(x))
    return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data = read_data('drive/MyDrive/Parkinson/filtered_data.csv', sep = ';')

In [ ]:
paper_aff = data[['Id', 'DN', 'AA', 'Y']].copy()
paper_aff.dropna(subset = ['AA'], inplace = True, axis = 0)
paper_aff.loc[:, 'AfId'] = paper_aff['AA'].apply(lambda x: x[0]['AfId'])
paper_aff.loc[:, 'DAfN'] = paper_aff['AA'].apply(lambda x: x[0]['DAfN'])
paper_aff.drop('AA', axis = 1, inplace = True)

In [ ]:
papers_info = pd.read_csv('drive/MyDrive/Parkinson/papers_info.csv', sep = ';')

In [ ]:
aff_info = pd.read_csv('drive/MyDrive/Parkinson/affiliations_info.csv', sep = ';')

In [ ]:
paper_aff = paper_aff.merge(papers_info[['Id', 'In-degree rank', 'weight']], on = 'Id').rename(columns = {'In-degree rank':'Paper rank'})
paper_aff = paper_aff.merge(aff_info[['AfId', 'In-degree rank']], on = 'AfId', how = 'left').rename(columns = {'In-degree rank':'Affiliation rank'})

In [ ]:
paper_aff

,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
0,2754967293,"Global, regional, and national burden of neuro...",2017,3.985476e+07,Auckland University of Technology,31,221,233.0
1,2112455323,MDS clinical diagnostic criteria for Parkinson...,2015,2.800294e+09,Montreal General Hospital,1,1563,22.0
2,2177834950,Exercise as medicine – evidence for prescribin...,2015,1.240557e+08,University of Copenhagen,199,11,234.0
3,2558041282,Gut Microbiota Regulate Motor Deficits and Neu...,2016,1.224118e+08,California Institute of Technology,2,500,107.0
4,2901519529,"Global, regional, and national burden of Parki...",2018,5.388228e+06,University of Rochester,6,459,78.0
...,...,...,...,...,...,...,...,...
39806,2547599175,Management of dysarthria associated with Parki...,2016,NaN,,210,0,NaN
39807,3112561736,Comparative Study of History Causes and Risk o...,2020,NaN,,210,0,NaN
39808,2895767795,Papel del personal de enfermería especializado...,2019,NaN,"Sección de Neurología, Complexo Hospitalario U...",210,0,NaN
39809,2619593042,熟地平颤颗粒结合重复经颅磁刺激治疗帕金森氏病的临床研究 Clinical Study on ...,2017,NaN,,210,0,NaN


In [ ]:
paper_aff.sort_values('Paper rank').head(10)

,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
1,2112455323,MDS clinical diagnostic criteria for Parkinson...,2015,2.800294e+09,Montreal General Hospital,1,1563,22.0
3,2558041282,Gut Microbiota Regulate Motor Deficits and Neu...,2016,1.224118e+08,California Institute of Technology,2,500,107.0
15,2584311212,Epidemiology of Parkinson's disease.,2017,2.800741e+09,Haukeland University Hospital,3,485,81.0
6,1920030402,MDS research criteria for prodromal Parkinson'...,2015,2.801533e+09,German Center for Neurodegenerative Diseases,4,473,37.0
12,2531872507,The epidemiology of Parkinson's disease: risk ...,2016,1.362000e+08,Harvard University,5,469,7.0
4,2901519529,"Global, regional, and national burden of Parki...",2018,5.388228e+06,University of Rochester,6,459,78.0
19,2621775373,Non-motor features of Parkinson disease.,2017,4.512925e+07,University College London,6,459,5.0
9,2082425146,"The Roles of PINK1, Parkin and Mitochondrial F...",2015,NaN,"Biochemistry Section, Surgical Neurology Branc...",7,458,NaN
22,2111216320,α-Synuclein strains cause distinct synucleinop...,2015,9.946410e+07,Katholieke Universiteit Leuven,8,439,9.0
14,1934236512,Gut microbiota are related to Parkinson's dise...,2015,NaN,Department of NeurologyHelsinki University Cen...,9,416,NaN


In [ ]:
for i in range(1, 11):
    display(paper_aff[paper_aff['Affiliation rank'] == i].sort_values(['Paper rank']).head(10))

,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
5,2986932567,"Identification of novel risk loci, causal insi...",2019,1.299303e+09,National Institutes of Health,12,373,1.0
152,2340446708,Genetics in Parkinson disease: Mendelian versu...,2016,1.299303e+09,National Institutes of Health,36,206,1.0
91,2972937106,The genetic architecture of Parkinson's disease,2020,1.299303e+09,National Institutes of Health,55,162,1.0
48,2887997888,Parkin and PINK1 mitigate STING-induced inflam...,2018,1.299303e+09,National Institutes of Health,81,132,1.0
69,2937319110,Parkinson's disease age at onset genome-wide a...,2019,1.299303e+09,National Institutes of Health,97,114,1.0
204,2098399146,Diagnosis of Parkinson's disease on the basis ...,2015,1.299303e+09,National Institutes of Health,100,110,1.0
380,1518704304,Endogenous Parkin Preserves Dopaminergic Subst...,2015,1.299303e+09,National Institutes of Health,107,103,1.0
68,2972368830,Ageing as a risk factor for neurodegenerative ...,2019,1.299303e+09,National Institutes of Health,114,96,1.0
1266,2464353788,A New Glucocerebrosidase Chaperone Reduces α-S...,2016,1.299303e+09,National Institutes of Health,125,85,1.0
1213,2126909058,Meta-analyses on prevalence of selected Parkin...,2015,1.299303e+09,National Institutes of Health,136,74,1.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
29,2132310402,Prediagnostic presentations of Parkinson's dis...,2015,115076166.0,UCL Institute of Neurology,24,242,2.0
350,2129697505,Parkinson's disease dementia: a neural network...,2015,115076166.0,UCL Institute of Neurology,52,167,2.0
169,2549336896,Clinical variables and biomarkers in predictio...,2017,115076166.0,UCL Institute of Neurology,71,145,2.0
504,2805954533,α-synuclein oligomers interact with ATP syntha...,2018,115076166.0,UCL Institute of Neurology,89,122,2.0
621,2176451886,Alpha-Synuclein Oligomers Interact with Metal ...,2016,115076166.0,UCL Institute of Neurology,97,114,2.0
1171,2179197468,The Concept of Prodromal Parkinson's Disease.,2015,115076166.0,UCL Institute of Neurology,110,100,2.0
802,2039736612,Glucocerebrosidase and Parkinson disease: Rece...,2015,115076166.0,UCL Institute of Neurology,111,99,2.0
894,1985287196,The ongoing pursuit of neuroprotective therapi...,2015,115076166.0,UCL Institute of Neurology,115,95,2.0
1674,2528242093,Insulin resistance and Parkinson’s disease: A ...,2016,115076166.0,UCL Institute of Neurology,121,89,2.0
1285,2337871602,The relationship between glucocerebrosidase mu...,2016,115076166.0,UCL Institute of Neurology,129,81,2.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
183,2072675976,Lipid vesicles trigger α-synuclein aggregation...,2015,241749.0,University of Cambridge,49,176,3.0
192,2775514330,Structural basis of membrane disruption and ce...,2017,241749.0,University of Cambridge,74,142,3.0
837,2176767889,Baseline and longitudinal grey matter changes ...,2015,241749.0,University of Cambridge,91,120,3.0
163,2588008972,Impulse control disorders and levodopa-induced...,2017,241749.0,University of Cambridge,102,108,3.0
379,1840177221,Cell‑based therapies for Parkinson disease —pa...,2015,241749.0,University of Cambridge,103,107,3.0
600,2571594992,A natural product inhibits the initiation of α...,2017,241749.0,University of Cambridge,116,94,3.0
1103,1833758113,"King's Parkinson's disease pain scale, the fir...",2015,241749.0,University of Cambridge,116,94,3.0
921,2512757939,Mutations associated with familial Parkinson’s...,2016,241749.0,University of Cambridge,120,90,3.0
761,2443158342,Chemical properties of lipids strongly affect ...,2016,241749.0,University of Cambridge,121,89,3.0
949,2233667077,Serum immune markers and disease progression i...,2016,241749.0,University of Cambridge,128,82,3.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
283,2021439462,Mitochondrial dysfunction and mitophagy in Par...,2015,40120149.0,University of Oxford,39,199,4.0
351,2616733389,The Role of Astrocyte Dysfunction in Parkinson...,2017,40120149.0,University of Oxford,70,146,4.0
955,2277130721,ER Stress and Autophagic Perturbations Lead to...,2016,40120149.0,University of Oxford,83,130,4.0
209,2599888381,The modulatory effect of adaptive deep brain s...,2017,40120149.0,University of Oxford,94,117,4.0
87,2095134828,Detecting and monitoring the symptoms of Parki...,2015,40120149.0,University of Oxford,100,110,4.0
178,1624016815,Bilateral adaptive deep brain stimulation is e...,2016,40120149.0,University of Oxford,102,108,4.0
736,2745164635,Alpha-synuclein oligomers: a new hope,2017,40120149.0,University of Oxford,104,106,4.0
465,2329808372,Direct visualization of alpha-synuclein oligom...,2015,40120149.0,University of Oxford,108,102,4.0
610,2764335780,Beta burst dynamics in Parkinson's disease OFF...,2017,40120149.0,University of Oxford,116,94,4.0
2079,2032730800,Impaired intracellular trafficking defines ear...,2015,40120149.0,University of Oxford,121,89,4.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
19,2621775373,Non-motor features of Parkinson disease.,2017,45129253.0,University College London,6,459,5.0
25,2744843428,Exenatide once weekly versus placebo in Parkin...,2017,45129253.0,University College London,44,186,5.0
289,2474900144,Visual dysfunction in Parkinson's disease.,2016,45129253.0,University College London,81,132,5.0
631,2025843054,Evolution of prodromal clinical markers of Par...,2015,45129253.0,University College London,99,111,5.0
75,2138685200,Lewy body dementias,2015,45129253.0,University College London,117,93,5.0
426,2774903942,Investigating the genetic architecture of deme...,2018,45129253.0,University College London,139,71,5.0
1775,2474991972,Autophagic lysosome reformation dysfunction in...,2016,45129253.0,University College London,142,68,5.0
2648,2553724719,Ambroxol effects in glucocerebrosidase and α-s...,2016,45129253.0,University College London,142,68,5.0
574,2563082987,Assessment of Safety and Efficacy of Safinamid...,2017,45129253.0,University College London,144,66,5.0
754,2998953100,Ambroxol for the Treatment of Patients With Pa...,2020,45129253.0,University College London,145,65,5.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
74,2571740089,Selective neuronal vulnerability in Parkinson ...,2017,111979921.0,Northwestern University,14,309,6.0
55,2752710053,Dopamine oxidation mediates mitochondrial and ...,2017,111979921.0,Northwestern University,27,235,6.0
136,2587082207,α-synuclein toxicity in neurodegeneration: mec...,2017,111979921.0,Northwestern University,29,224,6.0
361,2261503230,α-Synuclein-induced lysosomal dysfunction occu...,2016,111979921.0,Northwestern University,70,146,6.0
366,808190111,Pioglitazone in early Parkinson's disease: a p...,2015,111979921.0,Northwestern University,104,106,6.0
1603,2763305669,Parkinson's Disease Is Not Simply a Prion Diso...,2017,111979921.0,Northwestern University,121,89,6.0
92,2416742466,Identification of TMEM230 mutations in familia...,2016,111979921.0,Northwestern University,126,84,6.0
1635,1546336211,Exercise improves cognition in Parkinson's dis...,2015,111979921.0,Northwestern University,130,80,6.0
2715,2779236324,Reversible Conformational Conversion of α-Synu...,2018,111979921.0,Northwestern University,133,77,6.0
2112,2903418495,"Synaptic, Mitochondrial, and Lysosomal Dysfunc...",2019,111979921.0,Northwestern University,135,75,6.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
12,2531872507,The epidemiology of Parkinson's disease: risk ...,2016,136199984.0,Harvard University,5,469,7.0
404,2083063722,Successful function of autologous iPSC-derived...,2015,136199984.0,Harvard University,118,92,7.0
104,2149200223,The PINK1-PARKIN Mitochondrial Ubiquitylation ...,2015,136199984.0,Harvard University,119,91,7.0
1228,1975932268,Progressive decline of glucocerebrosidase in a...,2015,136199984.0,Harvard University,122,88,7.0
1881,1767392319,Glucocerebrosidase gene therapy prevents α-syn...,2015,136199984.0,Harvard University,139,71,7.0
557,2521258590,Tau Positron Emission Tomographic Imaging in t...,2016,136199984.0,Harvard University,140,70,7.0
848,2771082365,α-synuclein Induces Mitochondrial Dysfunction ...,2018,136199984.0,Harvard University,150,60,7.0
764,1814421085,Nuclear receptor Nurr1 agonists enhance its du...,2015,136199984.0,Harvard University,152,58,7.0
2180,1913190924,Sustained Systemic Glucocerebrosidase Inhibiti...,2015,136199984.0,Harvard University,153,57,7.0
106,2951517287,Discovery and inhibition of an interspecies gu...,2019,136199984.0,Harvard University,154,56,7.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
81,2802062315,Cellular milieu imparts distinct pathological ...,2018,79576946.0,University of Pennsylvania,63,154,8.0
354,2513283541,A systematic review of the literature on disor...,2017,79576946.0,University of Pennsylvania,105,105,8.0
525,1414124913,Longitudinal study of normal cognition in Park...,2015,79576946.0,University of Pennsylvania,108,102,8.0
407,2330640534,CSF biomarkers associated with disease heterog...,2016,79576946.0,University of Pennsylvania,124,86,8.0
1666,2756232480,Dopamine induces soluble α-synuclein oligomers...,2017,79576946.0,University of Pennsylvania,128,82,8.0
1092,2521336258,Molecular and Biological Compatibility with Ho...,2016,79576946.0,University of Pennsylvania,143,67,8.0
825,2621873693,Impulse Control and Related Disorders in Parki...,2017,79576946.0,University of Pennsylvania,147,63,8.0
1640,2095868453,Incident impulse control disorder symptoms and...,2016,79576946.0,University of Pennsylvania,154,56,8.0
3004,1885676552,Rivastigmine for mild cognitive impairment in ...,2015,79576946.0,University of Pennsylvania,154,56,8.0
1080,2293549201,Longitudinal changes in cognition in early Par...,2016,79576946.0,University of Pennsylvania,155,55,8.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
22,2111216320,α-Synuclein strains cause distinct synucleinop...,2015,99464096.0,Katholieke Universiteit Leuven,8,439,9.0
587,2801720268,Linking Neuroinflammation and Neurodegeneratio...,2018,99464096.0,Katholieke Universiteit Leuven,98,113,9.0
752,2753111549,Cueing for people with Parkinson's disease wit...,2017,99464096.0,Katholieke Universiteit Leuven,145,65,9.0
688,2156580802,Feasibility and effects of home-based smartpho...,2016,99464096.0,Katholieke Universiteit Leuven,153,57,9.0
3640,2117786663,Microstructural changes in white matter associ...,2015,99464096.0,Katholieke Universiteit Leuven,153,57,9.0
4302,2022223015,Longitudinal follow-up and characterization of...,2015,99464096.0,Katholieke Universiteit Leuven,164,46,9.0
2415,2527849633,A LRRK2-Dependent EndophilinA Phosphoswitch Is...,2016,99464096.0,Katholieke Universiteit Leuven,164,46,9.0
1973,2608686748,Training dual tasks together or apart in Parki...,2017,99464096.0,Katholieke Universiteit Leuven,167,43,9.0
8631,2028586227,FK506 reduces neuroinflammation and dopaminerg...,2015,99464096.0,Katholieke Universiteit Leuven,169,41,9.0
1149,2805130622,Deficiency of parkin and PINK1 impairs age-dep...,2018,99464096.0,Katholieke Universiteit Leuven,169,41,9.0


,Id,DN,Y,AfId,DAfN,Paper rank,weight,Affiliation rank
61,1937033430,New Clinical Subtypes of Parkinson Disease and...,2015,28166907.0,Karolinska Institutet,25,239,10.0
1263,2120188418,The Effects of Highly Challenging Balance Trai...,2015,28166907.0,Karolinska Institutet,120,90,10.0
1953,2121103728,Eltoprazine counteracts l-DOPA-induced dyskine...,2015,28166907.0,Karolinska Institutet,125,85,10.0
787,2127508962,How to make a midbrain dopaminergic neuron,2015,28166907.0,Karolinska Institutet,125,85,10.0
50,2528543174,Molecular Diversity of Midbrain Development in...,2016,28166907.0,Karolinska Institutet,137,73,10.0
448,2335177398,Physical activity and risk of Parkinson’s dise...,2015,28166907.0,Karolinska Institutet,148,62,10.0
2603,1717096433,Cognitive impairment in Parkinson's disease an...,2016,28166907.0,Karolinska Institutet,149,61,10.0
678,2946151196,Evolution of prodromal Parkinson's disease and...,2019,28166907.0,Karolinska Institutet,153,57,10.0
2689,2098290220,Levels and Patterns of Physical Activity and S...,2015,28166907.0,Karolinska Institutet,159,51,10.0
1909,2183481716,Effect of the myeloperoxidase inhibitor AZD324...,2015,28166907.0,Karolinska Institutet,165,45,10.0


In [ ]:
paper_aff.to_csv('drive/MyDrive/paper_aff.csv', sep = ';')

In [ ]:
avg_cit = paper_aff.groupby(['AfId', 'DAfN', 'Affiliation rank']).mean()['weight'].reset_index().rename(columns = {'weight':'Average citation'})
avg_cit['Average citation rank'] = avg_cit['Average citation'].rank(method = 'dense', ascending = False)

In [ ]:
avg_cit = avg_cit.merge(paper_aff.groupby(['AfId', 'DAfN']).count()['Id'].reset_index().rename(columns = {'Id':'Paper count'}), on = ['AfId', 'DAfN'], how  = 'left')

In [ ]:
avg_cit.sort_values('Average citation rank').head(10)

,AfId,DAfN,Affiliation rank,Average citation,Average citation rank,Paper count
2845,2.801442e+09,Broomfield Hospital,205.0,279.000000,1.0,1
2760,2.800294e+09,Montreal General Hospital,22.0,230.000000,2.0,8
1359,1.224118e+08,California Institute of Technology,107.0,142.250000,3.0,4
2520,1.292975e+09,Ford Motor Company,335.0,101.000000,4.0,1
2996,3.018075e+09,Sanofi S.A.,353.0,77.000000,5.0,1
461,3.985476e+07,Auckland University of Technology,233.0,75.333333,6.0,3
2785,2.800741e+09,Haukeland University Hospital,81.0,75.200000,7.0,10
2709,2.799693e+09,Biotechnology and Biological Sciences Research...,358.0,69.000000,8.0,1
498,4.387913e+07,Government College University,360.0,67.000000,9.0,1
2049,1.839349e+08,Genentech,101.0,61.400000,10.0,10


In [ ]:
avg_cit.sort_values('Affiliation rank').head(10)

,AfId,DAfN,Affiliation rank,Average citation,Average citation rank,Paper count
2545,1.299303e+09,National Institutes of Health,1.0,14.929078,116.0,282
1268,1.150762e+08,UCL Institute of Neurology,2.0,17.874396,81.0,207
6,2.417490e+05,University of Cambridge,3.0,12.790614,158.0,277
464,4.012015e+07,University of Oxford,4.0,14.477477,121.0,222
516,4.512925e+07,University College London,5.0,16.427861,92.0,201
1228,1.119799e+08,Northwestern University,6.0,21.319444,59.0,144
1520,1.362000e+08,Harvard University,7.0,16.255952,97.0,168
882,7.957695e+07,University of Pennsylvania,8.0,12.301587,179.0,189
1101,9.946410e+07,Katholieke Universiteit Leuven,9.0,17.209677,85.0,124
336,2.816691e+07,Karolinska Institutet,10.0,10.293269,240.0,208


In [ ]:
aff_net = pd.read_csv('drive/MyDrive/Parkinson/aff_net.csv', sep = ';')

In [ ]:
aff_net_no_periods = aff_net.groupby(['AfId1', 'AfId2']).sum()['weight'].reset_index()

In [ ]:
aff_net_no_periods.to_csv('aff_net_no_periods.csv', sep = '\t', index = False, header = False)

In [ ]:
len(set(aff_net_no_periods['AfId1'].values) | set(aff_net_no_periods['AfId2'].values))

3029

In [ ]:
aff_net_no_periods

,AfId1,AfId2,weight
0,4.605000e+03,1.380062e+08,1
1,9.507000e+03,9.507000e+03,1
2,9.507000e+03,3.592860e+07,2
3,9.507000e+03,4.012015e+07,1
4,9.507000e+03,1.776393e+08,1
...,...,...,...
128597,3.148998e+09,1.839358e+08,1
128598,3.148998e+09,1.869036e+08,2
128599,3.148998e+09,8.638962e+08,1
128600,3.148998e+09,1.299303e+09,2
